On télécharge et nettoie ici les données de la police entre 1996 et 2022, récupérées sur le site du ministère de l'intérieur

In [1]:
import requests
import pandas as pd
import os

In [2]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
# URL du fichier Excel à télécharger
url = "https://static.data.gouv.fr/resources/chiffres-departementaux-mensuels-relatifs-aux-crimes-et-delits-enregistres-par-les-services-de-police-et-de-gendarmerie-depuis-janvier-1996/20221031-102847/tableaux-4001-ts.xlsx" 

# Dossier de destination pour enregistrer le fichier téléchargé
dossier_destination = "data_police"
os.makedirs(dossier_destination, exist_ok=True)

# Chemin du fichier téléchargé
fichier_telecharge = os.path.join(dossier_destination, "fichier.xlsx")

# Téléchargement du fichier
print("Téléchargement en cours...")
response = requests.get(url, stream=True)
with open(fichier_telecharge, "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)
print("Téléchargement terminé:", fichier_telecharge)

# Charger les onglets du fichier Excel en DataFrames
print("Chargement des onglets dans des DataFrames...")
data = pd.read_excel(fichier_telecharge, sheet_name=None)  # `sheet_name=None` charge tous les onglets

# Afficher les noms des onglets
print("Noms des onglets :", data.keys())

# Boucler sur chaque onglet et afficher les premières lignes
for nom_onglet, df in data.items():
    print(f"\nOnglet : {nom_onglet}")
    print(df.head())

# Fusionner tous les onglets dans un unique DataFrame avec une colonne identifiant l'onglet
dfs_combines = pd.concat(
    [df.assign(Onglet=nom_onglet) for nom_onglet, df in data.items()],
    ignore_index=True
)

# Afficher le DataFrame combiné
print("\nDataFrame combiné avec tous les onglets :")
print(dfs_combines)

# Sauvegarder le DataFrame combiné dans un fichier Excel
excel_output_path = os.path.join(dossier_destination, "data_combines.xlsx")
dfs_combines.to_excel(excel_output_path, index=False)

print(f"Le DataFrame combiné a été sauvegardé dans : {excel_output_path}")

# Supprimer le fichier téléchargé si ce n'est plus nécessaire
os.remove(fichier_telecharge)
print("Fichier téléchargé supprimé.")

Téléchargement en cours...


Téléchargement terminé: data_police/fichier.xlsx
Chargement des onglets dans des DataFrames...
Noms des onglets : dict_keys(['France_Entière', 'France_Métro', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2A', '2B', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '971', '972', '973', '974', '975', '976', '977', '978', '986', '987', '988'])

Onglet : France_Entière
   Index                                      libellé index  _2022_08  \
0      1             Règlements de compte entre malfaiteurs        14   
1      2       H

On sépare le datadrame combiné en deux pour obtenir une base de données départementale et une base de données pour la france entière


In [4]:
#Données pour la france entière
df_france = dfs_combines[dfs_combines['Onglet'].isin(['France_Entière', 'France_Métro'])]

# Afficher le DataFrame pour violence et meurtre
print("DataFrame la France entière:")
print(df_france)

# Sauvegarder le DataFrame dans un fichier Excel
excel_output_path_fr = os.path.join(dossier_destination, "data_france.xlsx")
df_france.to_excel(excel_output_path_fr, index=False)

print(f"Le DataFrame combiné a été sauvegardé dans : {excel_output_path_fr}")

DataFrame la France entière:
     Index                                      libellé index  _2022_08  \
0        1             Règlements de compte entre malfaiteurs        14   
1        2       Homicides pour voler et à l'occasion de vols         2   
2        3                     Homicides pour d'autres motifs        88   
3        4  Tentatives d'homicides pour voler et à l'occas...        15   
4        5          Tentatives homicides pour d'autres motifs       315   
..     ...                                                ...       ...   
209    103  Infractions à l'exercice d'une profession règl...       130   
210    104  Infractions au droit de l'urbanisme et de la c...       328   
211    105                                   Fraudes fiscales        62   
212    106            Autres délits économiques et financiers      1168   
213    107                                      Autres délits     13941   

     _2022_07  _2022_06  _2022_05  _2022_04  _2022_03  _2022_02  _2022

Le DataFrame combiné a été sauvegardé dans : data_police/data_france.xlsx


In [5]:
# Sélectionner les lignes où 'Onglet' n'est ni 'France_Entière' ni 'France_Métro'
df_dep = dfs_combines[~dfs_combines['Onglet'].isin(['France_Entière', 'France_Métro'])]

# Afficher le DataFrame pour les lignes qui ne sont ni 'France_Entière' ni 'France_Métro'
print("DataFrame sans France entière ni France Métro :")
print(df_dep)

# Sauvegarder le DataFrame dans un fichier Excel
excel_output_path_fr = os.path.join(dossier_destination, "data_dep.xlsx")
df_dep.to_excel(excel_output_path_fr, index=False)

print(f"Le DataFrame combiné a été sauvegardé dans : {excel_output_path_fr}")


DataFrame sans France entière ni France Métro :
       Index                                      libellé index  _2022_08  \
214        1             Règlements de compte entre malfaiteurs         0   
215        2       Homicides pour voler et à l'occasion de vols         0   
216        3                     Homicides pour d'autres motifs         0   
217        4  Tentatives d'homicides pour voler et à l'occas...         0   
218        5          Tentatives homicides pour d'autres motifs         0   
...      ...                                                ...       ...   
11658    103  Infractions à l'exercice d'une profession règl...         3   
11659    104  Infractions au droit de l'urbanisme et de la c...         2   
11660    105                                   Fraudes fiscales         0   
11661    106            Autres délits économiques et financiers         6   
11662    107                                      Autres délits        66   

       _2022_07  _2022_06  

In [6]:
# Transformer le tableau en format "long"
df_france_long = df_france.melt(id_vars=['Index', 'libellé index', 'Onglet'],  # Colonnes fixes
                  var_name='Date',  # Nom pour la colonne de dates
                  value_name='Nombre')  # Nom pour la colonne des valeurs

# Afficher le résultat
print(df_france_long)

# Sauvegarder le DataFrame dans un fichier Excel
excel_output_path = os.path.join(dossier_destination, "data_france_long.xlsx")
df_france_long.to_excel(excel_output_path, index=False)

print(f"Le DataFrame a été sauvegardé dans : {excel_output_path}")

       Index                                      libellé index  \
0          1             Règlements de compte entre malfaiteurs   
1          2       Homicides pour voler et à l'occasion de vols   
2          3                     Homicides pour d'autres motifs   
3          4  Tentatives d'homicides pour voler et à l'occas...   
4          5          Tentatives homicides pour d'autres motifs   
...      ...                                                ...   
68475    103  Infractions à l'exercice d'une profession règl...   
68476    104  Infractions au droit de l'urbanisme et de la c...   
68477    105                                   Fraudes fiscales   
68478    106            Autres délits économiques et financiers   
68479    107                                      Autres délits   

               Onglet      Date  Nombre  
0      France_Entière  _2022_08    14.0  
1      France_Entière  _2022_08     2.0  
2      France_Entière  _2022_08    88.0  
3      France_Entière  _202